In [1]:
import pandas as pd
from itertools import combinations
import math
import os
import csv
import time

In [2]:
cargo_capacity_psv = 100
psv_speed = 10 #7 for strøm, 10 på resten
max_platforms_in_one_voyage = 6

mappenavn = '10'
confignr = 3
capacity = 70
idle_restriction = True

if capacity == 70: 
    capacity_fil = '70' # 70, 90 eller ''
    maxcap = 71
    mincap = 42
elif capacity == 90:
    capacity_fil = '90'
    maxcap = 91
    mincap = 54
elif capacity == 100: 
    capacity_fil = ''
    maxcap = 101
    mincap = 60

## Reads input data

In [3]:
demand = pd.read_csv(f'config{confignr}/{mappenavn}/output_platforms_demand.csv', header=0, delimiter=';')
distances = pd.read_csv(f'config{confignr}/{mappenavn}/output_distance_matrix_kmeans.csv', header=0, delimiter=';', index_col='from/to')

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['MON'] + demand['platform'].tolist() + ['MON']  # Add 'Mon' as start and end platforms

## Algorithm for generating the shortest routes

In [4]:
shortest_routes_dict = {}

def generate_routes(demand, distances):
    cargo_capacity = cargo_capacity_psv
    max_platforms = max_platforms_in_one_voyage + 2 

    def dp(platform, cargo_remaining, route, visited):
        if cargo_remaining < 0:
            return
        if len(route) > max_platforms:
            return
        if platform == 'MON' and len(route) > 2:
            total_demand = sum(platforms_demand[p] for p in route[1:-1])
            if total_demand <= cargo_capacity:
                key = tuple(sorted(set(route)))
                total_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
                if key not in shortest_routes_dict or total_distance < shortest_routes_dict[key][1]:
                    duration_sailing = round((total_distance / psv_speed), 2)
                    duration_lossing = round(((total_demand * 1.389) / psv_speed), 2)
                    duration_sailing = round(duration_sailing, 2)
                    duration_lossing = round(duration_lossing, 2)
                    shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            return

        # Check if the current route is dominated
        current_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
        current_demand = sum(platforms_demand[p] for p in route[1:-1])
        
        # Check for dominance in existing routes
        for key, (existing_route, existing_distance, existing_demand, _, _) in shortest_routes_dict.items():
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return
        
        # Check for dominance in subsequent routes starting with the same platforms
        for existing_route in dominated_routes:
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return

        for next_platform in platforms_demand.keys():
            if next_platform != platform and next_platform not in visited:
                try:
                    distance_to_next = distances.loc[platform, next_platform]
                    new_cargo_remaining = cargo_remaining - platforms_demand[next_platform]
                    dp(next_platform, new_cargo_remaining, route + [next_platform], visited.union({next_platform}))
                except KeyError:
                    print("KeyError occurred for platform:", next_platform)
                    continue

    for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):
        for route_combination in combinations(platforms_demand.keys(), r - 2):
            route = ['MON'] + list(route_combination) + ['MON']
            dp('MON', cargo_capacity, route, {'MON'})

            # Add the current route to the dominated routes set
            dominated_routes.add(tuple(route))

    return shortest_routes_dict

# Keep track of dominated routes to skip future routes starting with them
dominated_routes = set()
shortest_routes_dict = generate_routes(demand, distances)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {round(distance,2)}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print(len(shortest_routes_dict))

Shortest Route: ['MON', 'APTOSS', 'MON'], Total Distance: 134.82, Total Demand: 30.0, Duration sailing: 13.48, Duration lossing: 4.17
Shortest Route: ['MON', 'ASLGFAGFBGFC', 'MON'], Total Distance: 156.56, Total Demand: 86.0, Duration sailing: 15.66, Duration lossing: 11.95
Shortest Route: ['MON', 'DAB', 'MON'], Total Distance: 85.82, Total Demand: 40.0, Duration sailing: 8.58, Duration lossing: 5.56
Shortest Route: ['MON', 'DSASTASTBSTC', 'MON'], Total Distance: 183.48, Total Demand: 90.25, Duration sailing: 18.35, Duration lossing: 12.54
Shortest Route: ['MON', 'DSS', 'MON'], Total Distance: 164.34, Total Demand: 29.0, Duration sailing: 16.43, Duration lossing: 4.03
Shortest Route: ['MON', 'KVBNLNVAL', 'MON'], Total Distance: 138.2, Total Demand: 38.0, Duration sailing: 13.82, Duration lossing: 5.28
Shortest Route: ['MON', 'MID', 'MON'], Total Distance: 156.6, Total Demand: 14.0, Duration sailing: 15.66, Duration lossing: 1.94
Shortest Route: ['MON', 'OSCOSE', 'MON'], Total Distance:

## Capacity/idle time checks

In [5]:
shortest_routes_dict_cap = {}

for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict.items():
    #if demand > 60 and demand < 101: # capacity for 100
    if demand > mincap and demand < maxcap:
        shortest_routes_dict_cap[key] = (route, distance, demand, duration_sailing, duration_lossing)

### If capacity/ idle time restriction

In [6]:

# Initialize dictionaries to store routes
shortest_routes_dict_cap_idle = {}
routes_to_remove = []

if idle_restriction == True:

    # Iterate through routes in shortest_routes_dict_cap
    for key, (route, distance, demand, duration_sailing, duration_lossing) in shortest_routes_dict_cap.items():
        total_duration = duration_sailing + duration_lossing
        if 16 < total_duration < 24 or 40 < total_duration < 48:
            # If the route meets the conditions, add it to the routes_to_remove list
            routes_to_remove.append(key)
        else:
            # If the route does not meet the conditions, add it to shortest_routes_dict_cap_idle
            shortest_routes_dict_cap_idle[key] = (route, distance, demand, duration_sailing, duration_lossing)

    # Remove routes that meet the conditions from shortest_routes_dict_cap
    for key in routes_to_remove:
        del shortest_routes_dict_cap[key]

    # Print counts
    print("Count of routes with duration between 16 and 24 removed:", len(routes_to_remove))
    print("Count of routes with duration between 40 and 48 removed:", len(routes_to_remove))

Count of routes with duration between 16 and 24 removed: 6
Count of routes with duration between 40 and 48 removed: 6


## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 2294 routes

In [7]:



routes_only = [route for route, _, _, _, _ in shortest_routes_dict_cap.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict_cap.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict_cap.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict_cap.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict_cap.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv(f'config{confignr}/{mappenavn}/distances{capacity_fil}.csv', index=False)
df_demand.to_csv(f'config{confignr}/{mappenavn}/demand{capacity_fil}.csv', index=False)
df_duration_sailing.to_csv(f'config{confignr}/{mappenavn}/duration_sailing{capacity_fil}.csv', index=False)
df_duration_lossing.to_csv(f'config{confignr}/{mappenavn}/duration_lossing{capacity_fil}.csv', index=False)
route_file = f"config{confignr}/{mappenavn}/routes{capacity_fil}.csv"

def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)